# Data Preparation

## Import data

In [16]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

### Ubereats

In [12]:
url = "https://raw.githubusercontent.com/cedricly-git/BADS_Capstone_repo/main/Data/ubereats+time_related_vars.csv"

In [13]:
uber = pd.read_csv(url)

In [14]:
uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Day                       728 non-null    object 
 1   trend_value               728 non-null    int64  
 2   weight                    728 non-null    float64
 3   total_ubereats            728 non-null    int64  
 4   estimated_daily_searches  728 non-null    int64  
 5   weekday                   728 non-null    object 
 6   is_weekend                728 non-null    int64  
 7   is_holiday                728 non-null    int64  
 8   month                     728 non-null    object 
 9   season                    728 non-null    object 
dtypes: float64(1), int64(5), object(4)
memory usage: 57.0+ KB


### Weather

In [ ]:
# Define parameters - Top 10 most populated cities in Switzerland
# Using population-weighted approach to represent country-wide behavior
cities = {
    "Zurich": {"lat": 47.3769, "lon": 8.5417, "pop": 436551},
    "Geneva": {"lat": 46.2044, "lon": 6.1432, "pop": 209061},
    "Basel": {"lat": 47.5596, "lon": 7.5886, "pop": 177571},
    "Lausanne": {"lat": 46.5197, "lon": 6.6323, "pop": 144873},
    "Bern": {"lat": 46.9481, "lon": 7.4474, "pop": 137995},
    "Winterthur": {"lat": 47.5056, "lon": 8.7247, "pop": 120376},
    "Lucerne": {"lat": 47.0502, "lon": 8.3064, "pop": 86234},
    "St. Gallen": {"lat": 47.4245, "lon": 9.3767, "pop": 78863},
    "Lugano": {"lat": 46.0101, "lon": 8.9600, "pop": 63629},
    "Biel": {"lat": 47.1404, "lon": 7.2471, "pop": 56896}
}

# Calculate population weights
total_pop = sum(city["pop"] for city in cities.values())
city_weights = {name: city["pop"] / total_pop for name, city in cities.items()}

start_date = "2023-11-13"
end_date = "2025-11-11"

weather_data = []

# Loop through cities
for city, coords in cities.items():
    url1 = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={coords['lat']}&longitude={coords['lon']}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum&timezone=Europe/Zurich"
    )
    data = requests.get(url1).json()
    for i, date in enumerate(data['daily']['time']):
        weather_data.append({
            "Day": datetime.strptime(date, "%Y-%m-%d"),
            "City": city,
            "Temp_Max": data['daily']['temperature_2m_max'][i],
            "Temp_Min": data['daily']['temperature_2m_min'][i],
            "Precipitation": data['daily']['precipitation_sum'][i]
        })

# Convert to DataFrame
weather_df = pd.DataFrame(weather_data)


In [30]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Day            2190 non-null   datetime64[ns]
 1   City           2190 non-null   object        
 2   Temp_Max       2190 non-null   float64       
 3   Temp_Min       2190 non-null   float64       
 4   Precipitation  2190 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 85.7+ KB


In [ ]:
# Convert UberEats Day column to datetime
uber['Day'] = pd.to_datetime(uber['Day'])

# Calculate population-weighted average weather for each day
# This represents country-wide weather based on major cities
weather_df['pop_weight'] = weather_df['City'].map(city_weights)

# Calculate weighted averages
weather_avg = weather_df.groupby('Day').apply(
    lambda x: pd.Series({
        'Temp_Max': (x['Temp_Max'] * x['pop_weight']).sum(),
        'Temp_Min': (x['Temp_Min'] * x['pop_weight']).sum(),
        'Precipitation': (x['Precipitation'] * x['pop_weight']).sum()
    })
).reset_index()

# Merge datasets using population-weighted averages
merged_df = pd.merge(uber, weather_avg, on='Day', how='left')

# Check the result
print(merged_df.head())
print(f"Merged dataset shape: {merged_df.shape}")

         Day  trend_value    weight  total_ubereats  estimated_daily_searches  \
0 2023-11-13           35  0.003880          236627                       918   
1 2023-11-14           52  0.005765          236676                      1364   
2 2023-11-15           74  0.008204          232238                      1905   
3 2023-11-16           44  0.004878          233488                      1139   
4 2023-11-17           53  0.005876          258156                      1517   

     weekday  is_weekend  is_holiday     month season    City  Temp_Max  \
0     Monday           0           0  November   Fall  Zurich      15.3   
1    Tuesday           0           0  November   Fall  Zurich      14.4   
2  Wednesday           0           0  November   Fall  Zurich      10.8   
3   Thursday           0           0  November   Fall  Zurich      11.1   
4     Friday           0           0  November   Fall  Zurich      10.7   

   Temp_Min  Precipitation  
0       9.6            7.4  
1   

## Feature Engineering

In [39]:
# Feature Engineering

# Convert "Day" to datetime format
merged_df["Day"] = pd.to_datetime(merged_df["Day"])

# Extract day of week and month number
merged_df["dayofweek"] = merged_df["Day"].dt.weekday    
merged_df["month_num"] = merged_df["Day"].dt.month

# Encode categorical variables
merged_df = pd.get_dummies(merged_df, columns=["season", "month"], drop_first=True)

In [40]:
# Define target and features
y = merged_df["total_ubereats"]

X = merged_df[[
    "is_weekend", 
    "is_holiday",
    "dayofweek",
    "Temp_Max",
    "Precipitation",
]]

## Linear Regression

In [41]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

coef = pd.DataFrame({
    "feature": X.columns,
    "coef": model.coef_
})

print(coef)


         feature          coef
0     is_weekend    429.077892
1     is_holiday -27361.902453
2      dayofweek    440.636699
3       Temp_Max   1087.662629
4  Precipitation  -2225.328800
